##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Análise de modelo usando o TFX Pipeline e o TensorFlow Model Analysis


Observação: recomendamos executar este tutorial em um notebook Colab, sem necessidade de configuração! Basta clicar em “Executar no Google Colab”.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfma"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/penguin_tfma.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/penguin_tfma.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/penguin_tfma.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

Neste tutorial baseado em notebook, criaremos e executaremos um pipeline TFX que cria um modelo de classificação simples e analisa seu desempenho em múltiplas execuções. Este notebook é baseado no pipeline TFX que construímos no [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple). Se você ainda não leu esse tutorial, leia-o antes de prosseguir com este notebook.

Ao ajustar seu modelo ou treiná-lo com um novo dataset, você precisa verificar se seu modelo melhorou ou piorou. Apenas verificar as métricas de nível superior, como a exatidão, pode não ser suficiente. Cada modelo treinado deve ser avaliado antes de ser enviado para produção.

Adicionaremos um componente `Evaluator` ao pipeline criado no tutorial anterior. O componente Evaluator realiza análises profundas de seus modelos e compara o novo modelo com uma referência para determinar se eles são "bons o suficiente". Ele é implementado usando a biblioteca [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/guide/tfma).

Veja [Introdução aos pipelines do TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para saber mais sobre vários conceitos do TFX.

## Configuração

O processo de configuração é igual ao do tutorial anterior.

Primeiro precisamos instalar o pacote TFX Python e baixar o dataset que usaremos para nosso modelo.

### Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instale o TFX


In [ ]:
!pip install -U tfx

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

### Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime clicando no botão "RESTART RUNTIME" acima ou usando o menu "Runtime &gt; Restart runtime ...". Isso é necessário devido à maneira como o Colab carrega os pacotes.

Verifique as versões do TensorFlow e TFX.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### Configuração de variáveis

Existem algumas variáveis ​​usadas para definir um pipeline. Você pode personalizar essas variáveis ​​como desejar. Por padrão, toda a saída do pipeline será gerada no diretório atual.

In [ ]:
import os

PIPELINE_NAME = "penguin-tfma"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Preparação dos dados de exemplo

Usaremos o mesmo [dataset Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

Existem quatro características numéricas neste dataset que já foram normalizados para ter intervalo [0,1]. Construiremos um modelo de classificação que prevê as `species` de pinguins.

Como o TFX ExampleGen lê entradas de um diretório, precisamos criar um diretório e copiar o dataset para ele.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

## Crie um pipeline

Adicionaremos um componente [`Evaluator`](https://www.tensorflow.org/tfx/guide/evaluator) ao pipeline que criamos no [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Um componente Evaluator requer dados de entrada de um componente `ExampleGen` e um modelo de um componente `Trainer` e um objeto [`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig). Opcionalmente, podemos fornecer um modelo de referência que pode ser usado para comparar métricas com o modelo recém-treinado.

Um evaluator cria dois tipos de artefatos de saída, `ModelEvaluation` e `ModelBlessing`. ModelEvaluation contém o resultado detalhado da avaliação que pode ser investigado e visualizado posteriormente com a biblioteca TFMA. ModelBlessing contém um resultado booleano se o modelo passou em determinados critérios e pode ser usado em componentes posteriores, como um Pusher, como um sinal.


### Escreva o código para treinamento do modelo

Usaremos o mesmo código de modelo do [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

### Escreva uma definição de pipeline

Definiremos uma função para criar um pipeline TFX. Além do componente Evaluator que mencionamos acima, adicionaremos mais um nó chamado [`Resolver`](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/dsl/Resolver). Para verificar se um novo modelo está ficando melhor que o modelo anterior, precisamos compará-lo com um modelo publicado anteriormente, chamado de referência (baseline). O [ML Metadata (MLMD)](https://www.tensorflow.org/tfx/guide/mlmd) rastreia todos os artefatos anteriores do pipeline e o `Resolver` pode encontrar qual foi o modelo *abençoado* mais recente - um modelo aprovado no Evaluator com sucesso - do MLMD usando uma classe de estratégia chamada `LatestBlessedModelStrategy`.


In [ ]:
import tensorflow_model_analysis as tfma

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # NEW: Get the latest blessed model for Evaluator.
  model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

  # NEW: Uses TFMA to compute evaluation statistics over features of a model and
  #   perform quality validation of a candidate model (compared to a baseline).

  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='species')],
      slicing_specs=[
          # An empty slice spec means the overall slice, i.e. the whole dataset.
          tfma.SlicingSpec(),
          # Calculate metrics for each penguin species.
          tfma.SlicingSpec(feature_keys=['species']),
          ],
      metrics_specs=[
          tfma.MetricsSpec(per_slice_thresholds={
              'sparse_categorical_accuracy':
                  tfma.PerSliceMetricThresholds(thresholds=[
                      tfma.PerSliceMetricThreshold(
                          slicing_specs=[tfma.SlicingSpec()],
                          threshold=tfma.MetricThreshold(
                              value_threshold=tfma.GenericValueThreshold(
                                   lower_bound={'value': 0.6}),
                              # Change threshold will be ignored if there is no
                              # baseline model resolved from MLMD (first run).
                              change_threshold=tfma.GenericChangeThreshold(
                                  direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                  absolute={'value': -1e-10}))
                       )]),
          })],
      )
  evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'], # Pass an evaluation result.
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      trainer,

      # Following two components were added to the pipeline.
      model_resolver,
      evaluator,

      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

Precisamos fornecer as seguintes informações ao Evaluator via `eval_config`:

- Métricas adicionais para configurar (se desejar mais métricas do que as definidas no modelo).
- Fatias para configurar
- Limites de validações do modelo para verificar se a validação deve ser incluída

Como o `SparseCategoricalAccuracy` já foi incluído na chamada `model.compile()`, ele será incluído na análise automaticamente. Portanto, não adicionamos nenhuma métrica adicional aqui. O `SparseCategoricalAccuracy` será usado para decidir se o modelo também é bom o suficiente.

Computamos as métricas para todo o dataset e para cada espécie de pinguim. `SlicingSpec` especifica como agregamos as métricas declaradas.

Existem dois limites que um novo modelo deve ultrapassar, um é um limite absoluto de 0,6 e o ​​outro é um limite relativo que deve ser superior ao modelo de referência. Ao executar o pipeline pela primeira vez, `change_threshold` será ignorado e apenas value_threshold será verificado. Se você executar o pipeline mais de uma vez, o `Resolver` encontrará um modelo da execução anterior que será usado como modelo de referência para comparação.

Consulte o [Guia do componente Evaluator](https://www.tensorflow.org/tfx/guide/evaluator#using_the_evaluator_component) para mais informações.

## Execute o pipeline


Usaremos o `LocalDagRunner` como no tutorial anterior.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

Quando o pipeline for concluído, você deverá ver algo como o seguinte:

```
INFO:absl:Blessing result True written to pipelines/penguin-tfma/Evaluator/blessing/4.
```

Ou você também pode verificar manualmente o diretório de saída onde os artefatos gerados estão armazenados. Se você visitar `pipelines/penguin-tfma/Evaluator/blessing/` com um navegador de arquivos, poderá ver um arquivo com o nome `BLESSED` (abençoado) ou `NOT_BLESSED` (não abençoado) de acordo com o resultado da avaliação.

Se o resultado da bênção for `False`, o Pusher se recusará a enviar o modelo para `serving_model_dir`, porque o modelo não é bom o suficiente para ser usado em produção.

Você pode executar o pipeline novamente, possivelmente com configurações de avaliação diferentes. Mesmo que você execute o pipeline exatamente com a mesma configuração e dataset, o modelo treinado poderá ser um pouco diferente devido à aleatoriedade inerente ao treinamento do modelo, que pode levar a um modelo `NOT_BLESSED`.

### Examine as saídas do pipeline

Você pode usar o TFMA para investigar e visualizar o resultado da avaliação no artefato ModelEvaluation.

> **OBSERVAÇÃO: Se você não estiver no Colab, instale o Jupyter Extensions.** Você precisa de uma extensão TensorFlow Model Analysis para ver a visualização do TFMA. Esta extensão já está instalada no Google Colab, mas pode ser necessário instalá-la se você estiver executando este notebook em outros ambientes. Consulte as instruções de instalação da extensão Jupyter no [guia de instalação](https://github.com/tensorflow/model-analysis#installation).


#### Obtenha o resultado da análise dos artefatos de saída

Você pode usar APIs MLMD para localizar essas saídas programaticamente. Primeiro, definiremos algumas funções utilitárias para procurar os artefatos de saída que acabaram de ser produzidos.

In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)


Podemos encontrar a execução mais recente do componente `Evaluator` e obter artefatos de saída dele.

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  evaluator_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                          'Evaluator')
  eval_artifact = evaluator_output[standard_component_specs.EVALUATION_KEY][0]

O `Evaluator` sempre retorna um artefato de avaliação e podemos visualizá-lo usando a biblioteca TensorFlow Model Analysis. Por exemplo, o código a seguir renderizará as métricas de precisão para cada espécie de pinguim.

In [ ]:
import tensorflow_model_analysis as tfma

eval_result = tfma.load_eval_result(eval_artifact.uri)
tfma.view.render_slicing_metrics(eval_result, slicing_column='species')

Se você escolher 'sparse_categorical_accuracy' na lista suspensa `Show`, poderá ver os valores de exatidão por espécie. Você pode querer adicionar mais fatias e verificar se o seu modelo é bom para todas as distribuições e se existe algum possível bias.

## Próximos passos

Saiba mais sobre análise de modelos no [Tutorial da biblioteca TensorFlow Model Analysis](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).

Você encontrará mais recursos em https://www.tensorflow.org/tfx/tutorials.

Veja [Introdução aos pipelines do TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para saber mais sobre vários conceitos do TFX.
